**CSCI 4831: Sabermetrics Final Project**  
Name: Adam Ten Hoeve

https://github.com/aaronaaeng/Sabermetrics-Final-Project

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read in all of the statcast data
# df2016 = pd.read_csv("2016/data2016.csv")

# april2017 = pd.read_csv("2017/dataApril2017.csv")
# may2017 = pd.read_csv("2017/dataMay2017.csv")
# june2017 = pd.read_csv("2017/dataJune2017.csv")
# july2017 = pd.read_csv("2017/dataJuly2017.csv")
# aug2017 = pd.read_csv("2017/dataAug2017.csv")
# sept2017 = pd.read_csv("2017/dataSept2017.csv")
# oct2017 = pd.read_csv("2017/dataOct2017.csv")

april2018 = pd.read_csv("2018/dataApril2018.csv")
# may2018 = pd.read_csv("2018/dataMay2018.csv")
# june2018 = pd.read_csv("2018/dataJune2018.csv")
# july2018 = pd.read_csv("2018/dataJuly2018.csv")
# aug2018 = pd.read_csv("2018/dataAug2018.csv")
# sept2018 = pd.read_csv("2018/dataSept2018.csv")
# oct2018 = pd.read_csv("2018/dataOct2018.csv")

# df2017 = pd.concat([april2017, may2017, june2017, july2017, aug2017, sept2017, oct2017])
# df2018 = pd.concat([april2018, may2018, june2018, july2018, aug2018, sept2018, oct2018])

# dfTotal = pd.concat([df2016, df2017, df2018])
dfTotal = april2018.copy()

In [3]:
# Clean the data. 
# Group by pitcher. Get rid of anyone who pitched less than 100 times in the three years.
dfTempPitch = dfTotal.groupby("player_name", as_index = False).count()
dfTempPitch = dfTempPitch.loc[dfTempPitch["index"] > 100]
# Then group by batter to get rid of any batters who batted less than 100 times.
dfTempBat = dfTotal.groupby("batter", as_index = False).count()
dfTempBat = dfTempBat.loc[dfTempBat["index"] > 100]

# Create a database of pitchers 
dfPitchers = pd.merge(dfTotal, dfTempPitch[["player_name"]], on = "player_name", how = "inner")
# Create a database of batters
dfBatters = pd.merge(dfTotal, dfTempBat[["batter"]], on = "batter", how = "inner")

# Create a final dataframe of the cleaned batter and pitcher data
dfFinal = pd.merge(dfPitchers, dfBatters[["index", "pitcher", "batter", "release_speed", "strikes", "balls"]], on = ["index", "pitcher", "batter", "release_speed", "strikes", "balls"], how = "inner")

In [4]:
# A list of events of the batter either striking out or hitting the ball into play.
events = ["strikeout", "strikeout_double_play", "single", "double", "triple", "home_run", "sac_bunt", "sac_fly", "field_out", "field_error", "double_play", "grounded_into_double_play", "sac_fly_double_play", "triple_play", "walk", "hit_by_pitch"]
# Get rid of all the rows of data that don't have one of the events listed.
Outcomes = dfFinal.loc[dfFinal["events"].isin(events)]

In [5]:
# Calculate the proportion of each type of event. Will be used to calculate the weight of certain outcomes.
OutcomesProp = Outcomes.groupby("events", as_index = False).count()
numEvents = len(Outcomes)
OutcomesProp["proportion"] = OutcomesProp["index"] / numEvents
OutcomesProp = OutcomesProp[["events", "index", "proportion"]]
OutcomesProp

,events,index,proportion
0,double,1194,0.047000
1,double_play,60,0.002362
2,field_error,228,0.008975
3,field_out,10343,0.407141
4,grounded_into_double_play,478,0.018816
5,hit_by_pitch,305,0.012006
6,home_run,750,0.029523
7,sac_bunt,55,0.002165
8,sac_fly,188,0.007400
9,sac_fly_double_play,3,0.000118


In [6]:
# Create a dictionary of all batters and pitchers
batterElos = {}
pitcherElos = {}

# ids = list(Outcomes["pitcher"].unique()) + list(Outcomes["batter"].unique())
for id in list(Outcomes["pitcher"].unique()):
    pitcherElos[id] = 1000
for id in list(Outcomes["batter"].unique()):
    batterElos[id] = 1000
    
# Create a list of pitcher-favored outcomes.
pitcherFavored = ["strikeout", "strikeout_double_play", "field_out", "double_play", "grounded_into_double_play", "sac_fly_double_play", "triple_play"]
# Create a list of batter favored outcomes, meaning they did not get out by the end of the play.
batterFavored = ["single", "double", "triple", "home_run", "field_error", "walk", "hit_by_pitch", "sac_bunt", "sac_fly"]

In [7]:
def PercentOnBase(row):
    if row["events"] in batterFavored:
        return 1
    return 0

# Find how often batters get on base
total = len(Outcomes)

np.sum(Outcomes.apply(PercentOnBase, axis = 1)) / total

0.34537868052275233

In [13]:
Outcomes["events"].unique()

array(['field_out', 'double', 'single', 'strikeout', 'walk',
       'hit_by_pitch', 'grounded_into_double_play', 'field_error',
       'triple', 'sac_fly', 'home_run', 'strikeout_double_play',
       'double_play', 'sac_bunt', 'sac_fly_double_play', 'triple_play'],
      dtype=object)

In [19]:
# Determines the weight on the elo based on the type of event.
# Used to balance the proportion of pitcher-wins to batter-wins.
# Otherwise pitchers win about 70% of the time and dominate the scores
events = ["strikeout", "strikeout_double_play", "single", "double", "triple", "home_run", "sac_bunt", "sac_fly", "field_out", "field_error", "double_play", "grounded_into_double_play", "sac_fly_double_play", "triple_play", "walk", "hit_by_pitch"]
def EloEventWeight(event):
    # If that event occured
    if event == "strikeout":
        # Return the prob of that event times the assigned weight based on impact
        # All weights have are *2 what they normally are to speed up processing by preventing floating point arithmetic.
        return OutcomesProp.loc[OutcomesProp["events"] == "strikeout"]["proportion"] * 2
    elif event == "field_out":
        return OutcomesProp.loc[OutcomesProp["events"] == "field_out"]["proportion"] * 1
    elif event == "double_play":
        return OutcomesProp.loc[OutcomesProp["events"] == "double_play"]["proportion"] * 4
    elif event == "grounded_into_double_play":
        return OutcomesProp.loc[OutcomesProp["events"] == "grounded_into_double_play"]["proportion"] * 4
    elif event == "strikeout_double_play":
        return OutcomesProp.loc[OutcomesProp["events"] == "strikeout_double_play"]["proportion"] * 4
    elif event == "triple_play":
        return OutcomesProp.loc[OutcomesProp["events"] == "triple_play"]["proportion"] * 6
    elif event == "sac_fly_double_play":
        return OutcomesProp.loc[OutcomesProp["events"] == "sac_fly_double_play"]["proportion"] * 2
    elif event == "single":
        return OutcomesProp.loc[OutcomesProp["events"] == "single"]["proportion"] * 2
    elif event == "double":
        return OutcomesProp.loc[OutcomesProp["events"] == "double"]["proportion"] * 4
    elif event == "triple":
        return OutcomesProp.loc[OutcomesProp["events"] == "triple"]["proportion"] * 6
    elif event == "home_run":
        return OutcomesProp.loc[OutcomesProp["events"] == "home_run"]["proportion"] * 8
    elif event == "sac_bunt":
        return OutcomesProp.loc[OutcomesProp["events"] == "sac_bunt"]["proportion"] * 2
    elif event == "sac_fly":
        return OutcomesProp.loc[OutcomesProp["events"] == "sac_fly"]["proportion"] * 2
    elif event == "field_error":
        return OutcomesProp.loc[OutcomesProp["events"] == "field_error"]["proportion"] * 2
    elif event == "walk":
        return OutcomesProp.loc[OutcomesProp["events"] == "walk"]["proportion"] * 2
    elif event == "hit_by_pitch":
        return OutcomesProp.loc[OutcomesProp["events"] == "hit_by_pitch"]["proportion"] * 2
    else:
        print(event)
        return 0.5

In [20]:
# Create a function to calculate the probability a pitcher wins the matchup
def ProbPitcherWins(pitcher, batter):
    # Get ranking of pitcher
    pitcherRank = pitcherElos[pitcher]
    # Get ranking of batter
    batterRank = batterElos[batter]
    # From elo rankings. Assumes a difference of 400 is the same as 1 player being 10 times as likely to win
    probPitcherWins = 1 / (1 + 10**((pitcherRank -  batterRank)/400))
    return probPitcherWins

In [21]:
# Create a function to update the scores of players based on the results of pitches
def UpdateElo(row):
    # Get the probability that the pitcher wins from the function above
    prob = ProbPitcherWins(row["pitcher"], row["batter"])
    # Update the ratings based on the outcome of the pitch
    # If pitcher "wins" the matchup
    if row["events"] in pitcherFavored:
        # Largest possible change in score is 32.
        # Shown 16 here because *2 in weight function to prevent float multiplication.
        pitcherElos[row["pitcher"]] = int(pitcherElos[row["pitcher"]] + (16 * EloEventWeight(row["events"]) * (1 - prob)))
        batterElos[row["batter"]] = int(batterElos[row["batter"]] + (16 * EloEventWeight(row["events"]) * (prob - 1)))
    # If the batter wins the matchup
    else:
        pitcherElos[row["pitcher"]] = int(pitcherElos[row["pitcher"]] + (16 * EloEventWeight(row["events"]) * (prob - 1)))
        batterElos[row["batter"]] = int(batterElos[row["batter"]] + (16 * EloEventWeight(row["events"]) * (1 - prob)))

In [22]:
Outcomes.apply(UpdateElo, axis = 1);

In [23]:
print(list(batterElos.values()))

[399, 176, 265, 37, 424, -104, 445, -173, 709, 493, 655, 288, 129, 146, -35, 305, 413, 337, -8, 765, 228, 709, -30, -32, 90, -312, -77, -26, 348, -141, 84, 357, 649, 756, 89, 250, 435, 77, 452, 322, 140, 103, 76, 377, 475, 134, 758, 181, 462, -31, 440, 774, 389, 65, 500, -17, -230, -3, 262, 315, 359, 529, 572, 396, 525, 337, 559, -33, -206, 437, 250, 613, 210, 58, 207, 409, 51, 568, 88, 277, -19, 620, 697, 679, 258, 450, 130, 360, 0, 450, 198, 537, 653, 154, 17, 519, 204, 81, 385, 594, 6, 716, 9, 145, 435, 316, 544, 431, 172, 146, 295, -20, -84, 737, 764, 377, 101, 719, 274, 106, 794, 717, 86, -194, 421, 895, 482, 308, 411, 272, 128, -20, 526, 475, 497, 30, 97, 213, -180, 512, 513, 602, 385, 432, 260, 460, 827, 346, -65, 184, 542, -20, 755, 670, 201, 356, -192, -98, 550, 24, 32, 346, 747, 849, 244, 701, 681, -62, 203, 636, 490, 278, 127, -168, 98, 450, -29, 768, 683, 254, 593, 386, -21, 288, 387, 231, 387, 696, 308, 277, 262, 711, 245, 177, 124, -6, 284, -87, 41, -43, 290, 630, 707, 41

In [24]:
print(list(pitcherElos.values()))

[1194, 1320, 1399, 1339, 1259, 1337, 1754, 1381, 1294, 1199, 1223, 1366, 1206, 1035, 2325, 1908, 1340, 1161, 1138, 1209, 1801, 1325, 1535, 1271, 1367, 1342, 1174, 1349, 1422, 1041, 1220, 1334, 1146, 1254, 1174, 1982, 1817, 1209, 1354, 1322, 1257, 1179, 1283, 1649, 1344, 1200, 1298, 1379, 2226, 1201, 1575, 1617, 1385, 1319, 1193, 1327, 1290, 1327, 1273, 1638, 1112, 1270, 1347, 1430, 1219, 1236, 1362, 1834, 1419, 1415, 1244, 1567, 1192, 2112, 1739, 1196, 1152, 1529, 1232, 1804, 2020, 1325, 1267, 1164, 1982, 1158, 1305, 1192, 2329, 1146, 1101, 1371, 1337, 1095, 1338, 1217, 1684, 1535, 1174, 1365, 1274, 1302, 1241, 1623, 1390, 1210, 1352, 1301, 1575, 1184, 2622, 1357, 1323, 1343, 1385, 1730, 1765, 1518, 1217, 1254, 1411, 1194, 1513, 1504, 1439, 1318, 1252, 1479, 1288, 1641, 1841, 1415, 1211, 1230, 1262, 1545, 1461, 1336, 1442, 1287, 1237, 1946, 1355, 1398, 1185, 1348, 2063, 1582, 1286, 1826, 1156, 2159, 1055, 1260, 1109, 1192, 1197, 1200, 1522, 1751, 1138, 1357, 1220, 1192, 1021, 1564, 173

In [14]:
# Create dictionaries to convert player names and ids
IDtoName = {}
NametoID = {}
# Get all the unique players
temp = Outcomes[["player_name", "pitcher"]].groupby(by = "pitcher", as_index = False).first()

for i in range(len(temp)):
    IDtoName[temp["pitcher"].iloc[i]] = temp["player_name"].iloc[i]
    NametoID[temp["player_name"].iloc[i]] = temp["pitcher"].iloc[i]

In [13]:
# Create a dictionary to convert the player ID's to names
def PlayerIDtoName(id):
    return(IDtoName[id])

# Create a function to convert player's name to ID
def PlayerNametoID(name):
    return(NametoID[name])

In [ ]:
print("once")